# LangGraph 101 Typescript

[LLMs](https://js.langchain.com/docs/modules/models/chat/) make it possible to embed intelligence into a new class of applications. [LangGraph](https://langchain-ai.github.io/langgraphjs/) is a TypeScript/JavaScript framework to help build applications with LLMs. Here, we will overview the basics of LangGraph, explain its benefits, show how to use it to build workflows and agents, and show how it works with [LangChain JS](https://js.langchain.com/) and [LangSmith](https://smith.langchain.com/).

![ecosystem](img/ecosystem.png)

## Chat models

[Chat models](https://js.langchain.com/docs/modules/models/chat/) are the foundation of LLM applications. In TypeScript, chat models are accessed via standardized interfaces that take an array of message objects and return a response message. LangChain JS provides [a unified interface for chat models](https://js.langchain.com/docs/modules/models/chat/), making it easy to [access many different providers](https://js.langchain.com/docs/integrations/providers/).

In [1]:
// LLM Initialization
// Set your OpenAI API key in your environment or .env file
const OPENAI_API_KEY = process.env.OPENAI_API_KEY;
if (!OPENAI_API_KEY) {
  throw new Error("OPENAI_API_KEY is not set in the environment.");
}

In [2]:
import { z } from "zod";
import { tool } from "@langchain/core/tools";
import { StateGraph, START, END, interrupt, Command } from "@langchain/langgraph";
import { initChatModel } from "langchain/chat_models/universal";

import "@langchain/langgraph/zod";
// Initialize the chat model (OpenAI GPT-4.1)
const llm = await initChatModel("openai:gpt-4.1", { temperature: 0.0 });

## Running the model

The `initChatModel` interface in LangChain JS provides [standardized methods](https://js.langchain.com/docs/concepts/runnables/) for using chat models, which include:
- `invoke()`: Synchronously process inputs and return outputs
- `stream()`: Return outputs [incrementally](https://js.langchain.com/docs/concepts/streaming/) as they're generated

In [3]:
// Run the model
const result = await llm.invoke("What is LangGraph?");

In [4]:
// Check the result state
console.log("Result type:", result); 

Result type: AIMessage {
  "id": "chatcmpl-BWd6YdKnYeiRESZcxMtnC7wYiKFMb",
  "content": "**LangGraph** is an open-source framework for building stateful, multi-agent applications using Large Language Models (LLMs). It is developed by the creators of LangChain and is designed to help developers orchestrate complex workflows involving multiple LLM agents that can interact, share memory, and make decisions based on the evolving state of a conversation or process.\n\n### Key Features of LangGraph\n\n- **Stateful Workflows:** Unlike simple chains or pipelines, LangGraph allows you to define and manage the state throughout the execution of your application, enabling more complex and context-aware interactions.\n- **Multi-Agent Collaboration:** You can define multiple agents (each powered by an LLM or other logic) that can communicate, collaborate, and make decisions together.\n- **Graph-Based Architecture:** Workflows are defined as graphs, where nodes represent agents or functions, and edge

In [5]:
// Print the result content
console.log(result);

AIMessage {
  "id": "chatcmpl-BWd6YdKnYeiRESZcxMtnC7wYiKFMb",
  "content": "**LangGraph** is an open-source framework for building stateful, multi-agent applications using Large Language Models (LLMs). It is developed by the creators of LangChain and is designed to help developers orchestrate complex workflows involving multiple LLM agents that can interact, share memory, and make decisions based on the evolving state of a conversation or process.\n\n### Key Features of LangGraph\n\n- **Stateful Workflows:** Unlike simple chains or pipelines, LangGraph allows you to define and manage the state throughout the execution of your application, enabling more complex and context-aware interactions.\n- **Multi-Agent Collaboration:** You can define multiple agents (each powered by an LLM or other logic) that can communicate, collaborate, and make decisions together.\n- **Graph-Based Architecture:** Workflows are defined as graphs, where nodes represent agents or functions, and edges represent p

In [6]:
// Extract the result content
console.log(result.content);

**LangGraph** is an open-source framework for building stateful, multi-agent applications using Large Language Models (LLMs). It is developed by the creators of LangChain and is designed to help developers orchestrate complex workflows involving multiple LLM agents that can interact, share memory, and make decisions based on the evolving state of a conversation or process.

### Key Features of LangGraph

- **Stateful Workflows:** Unlike simple chains or pipelines, LangGraph allows you to define and manage the state throughout the execution of your application, enabling more complex and context-aware interactions.
- **Multi-Agent Collaboration:** You can define multiple agents (each powered by an LLM or other logic) that can communicate, collaborate, and make decisions together.
- **Graph-Based Architecture:** Workflows are defined as graphs, where nodes represent agents or functions, and edges represent possible transitions based on the current state.
- **Integration with LangChain:** 

## Tools

[Tools](https://js.langchain.com/docs/concepts/tools/) are utilities that can be called by a chat model. In LangChain JS, you create tools using the `tool` function from `@langchain/core/tools`, often with Zod schemas for runtime validation. This approach automatically infers the tool's name, description, and expected arguments from the function definition. You can also use [Model Context Protocol (MCP) servers](https://github.com/langchain-ai/langchain-mcp-adapters) as LangChain-compatible tools in TypeScript.

In [35]:
import { tool } from "@langchain/core/tools";
import { z } from "zod";

// Define the write_email tool using LangChain's tool function and Zod for validation
 const writeEmail = tool(async ({ to, subject, content }) => {
  // Placeholder response - in real app would send email
  return `Email sent to ${to} with subject '${subject}' and content: ${content}`;
},{
  name: "write_email",
  description: "Write and send an email.",
  schema: z.object({
    to: z.string(),
    subject: z.string(),
    content: z.string(),
  }),
  
});

In [36]:
// Check the tool type
console.log("Tool type:", writeEmail)

Tool type: DynamicStructuredTool {
  lc_serializable: false,
  lc_kwargs: {
    name: 'write_email',
    description: 'Write and send an email.',
    schema: ZodObject {
      spa: [Function: bound safeParseAsync] AsyncFunction,
      _def: [Object],
      parse: [Function: bound parse],
      safeParse: [Function: bound safeParse],
      parseAsync: [Function: bound parseAsync] AsyncFunction,
      safeParseAsync: [Function: bound safeParseAsync] AsyncFunction,
      refine: [Function: bound refine],
      refinement: [Function: bound refinement],
      superRefine: [Function: bound superRefine],
      optional: [Function: bound optional],
      nullable: [Function: bound nullable],
      nullish: [Function: bound nullish],
      array: [Function: bound array],
      promise: [Function: bound promise],
      or: [Function: bound or],
      and: [Function: bound and],
      transform: [Function: bound transform],
      brand: [Function: bound brand],
      default: [Function: bound def

In [37]:
// Log the argument names of the writeEmail tool
console.log("writeEmail argument names:", Object.keys(writeEmail.schema.shape));

writeEmail argument names: [ 'to', 'subject', 'content' ]


In [38]:
// Tool description
console.log("Tool description:", writeEmail.description);

Tool description: Write and send an email.


## Tool Calling

Tools can be [called](https://js.langchain.com/docs/concepts/tool-calling/) by LLMs. When a tool is bound to the model, the model can choose to call the tool by returning a structured output with tool arguments. In TypeScript, use the `bindTools` method to augment an LLM with tools.

![tool-img](img/tool_call_detail.png)

You can use the [`toolChoice` parameter](https://js.langchain.com/docs/how_to/tool_choice/) to enforce tool calling behavior.

In [41]:
// Connect tools to a chat model
const modelWithTools = llm.bindTools([writeEmail], { toolChoice: "required", parallelToolCalls: false });

// The model will now be able to call tools
const output = await modelWithTools.invoke(
  "Please use the write_email tool to draft a response to my boss about tomorrow's meeting. The email should confirm my attendance and ask if there's anything specific I should prepare."
);

// Extract tool calls and execute them
if (output.tool_calls && output.tool_calls.length > 0) {
  const args = output.tool_calls[0].args;
  // Print the arguments in the desired format
  console.log(args);
 

  // Call the tool
  const result = await writeEmail.invoke(args);
  console.log(result); // e.g. "Email to boss@company.com drafted with subject 'Re: Meeting Tomorrow'"
} else {
  console.log("No tool calls found in output.");
}

{
  to: "[Your Boss's Email]",
  subject: "Re: Tomorrow's Meeting",
  content: "Dear [Boss's Name],\n" +
    '\n' +
    "Thank you for the invitation to tomorrow's meeting. I am confirming my attendance and look forward to participating.\n" +
    '\n' +
    'Please let me know if there is anything specific you would like me to prepare or bring to the meeting.\n' +
    '\n' +
    'Best regards,\n' +
    '[Your Name]'
}
Email sent to [Your Boss's Email] with subject 'Re: Tomorrow's Meeting' and content: Dear [Boss's Name],

Thank you for the invitation to tomorrow's meeting. I am confirming my attendance and look forward to participating.

Please let me know if there is anything specific you would like me to prepare or bring to the meeting.

Best regards,
[Your Name]


Above, we enforce tool calling by setting `toolChoice: "required"`, so the model will always call a tool to write an email.

![basic_prompt](img/tool_call.png)

## Workflows

There are many patterns for building applications with LLMs.

[You can embed LLM calls into pre-defined workflows](https://langchain-ai.github.io/langgraphjs/tutorials/workflows/), giving the system more agency to make decisions.

As an example, you could add a router step to determine whether to write an email or not.

![workflow_example](img/workflow_example.png)

## Agents

You can further increase agency, allowing the LLM to dynamically direct its own tool usage.

[Agents](https://langchain-ai.github.io/langgraphjs/tutorials/workflows/) are typically implemented as tool calling in a loop, where the output of each tool call is used to inform the next action.

![agent_example](img/agent_example.png)

Agents are well suited to open-ended problems where it's difficult to predict the *exact* steps needed in advance.

Workflows are often appropriate when the control flow can easily be defined in advance.

![workflow_v_agent](img/workflow_v_agent.png)

## What is LangGraph?

[LangGraph](https://langchain-ai.github.io/langgraphjs/concepts/high_level/) provides low-level supporting infrastructure that sits underneath *any* workflow or agent.

It does not abstract prompts or architecture, and provides a few benefits:

- **Control**: Make it easy to define and/or combine agents and workflows.
- **Persistence**: Provide a way to persist the state of a graph, which enables both memory and human-in-the-loop.
- **Testing, Debugging, and Deployment**: Provide an easy onramp for testing, debugging, and deploying applications.

### Control

LangGraph lets you define your application as a graph with:

1. *State*: What information do we need to track over the course of the application?
2. *Nodes*: How do we want to update this information over the course of the application?
3. *Edges*: How do we want to connect these nodes together?

You can use the [`StateGraph` class](https://langchain-ai.github.io/langgraphjs/concepts/low_level/) to initialize a LangGraph graph with a [Zod schema](https://langchain-ai.github.io/langgraphjs/how-tos/define-state/) for your state.

`State` defines the schema for information you want to track over the course of the application.

In TypeScript, this is typically a Zod object schema, which provides runtime validation and type safety.

In [45]:
import { StateGraph, START, END } from "@langchain/langgraph";
import { z } from "zod";

// Define the state schema using Zod
const StateSchema = z.object({
  request: z.string(),
  email: z.string(),
});



// Initialize the workflow graph
const workflow = new StateGraph(StateSchema);

Each node is simply a TypeScript function. This gives you full control over the logic inside each node.

Nodes receive the current state and return an object to update the state.

By default, [state keys are overwritten](https://langchain-ai.github.io/langgraphjs/how-tos/state-reducers/).

However, you can [define custom update logic (reducers)](https://langchain-ai.github.io/langgraphjs/concepts/low_level/#reducers) to control how state is merged or updated.

![nodes_edges](img/nodes_edges.png)

In [59]:
const writeEmailNode = async (state) => {
  // Use a more explicit prompt to encourage tool use
  const prompt = "Please use the write_email tool to draft a response to my boss about tomorrow's meeting. The email should confirm my attendance and ask if there's anything specific I should prepare.";
  const output = await modelWithTools.invoke(prompt);



  const args = output.tool_calls?.[0]?.args;
  let email = "";
  if (args) {
    email = await writeEmail.func(args);
  }
  return { email };
};

Edges connect nodes together. 

We specify the control flow by adding edges and nodes to our state graph. 

In [61]:
import { StateGraph, START, END } from "@langchain/langgraph";

// Define your state schema and node functions first
const workflow = new StateGraph(StateSchema);

workflow
  .addNode("write_email_node", writeEmailNode)
  .addEdge(START, "write_email_node")
  .addEdge("write_email_node", END);

// Only now, compile the workflow
const app = workflow.compile();

In [62]:
// Run the workflow
const workflowResult = await app.invoke({ request: "Draft a response to my boss about tomorrow's meeting" });
console.log(workflowResult);

{
  request: "Draft a response to my boss about tomorrow's meeting",
  email: "Email sent to [Boss's Email] with subject 'Confirmation of Attendance for Tomorrow's Meeting' and content: Dear [Boss's Name],\n" +
    '\n' +
    "I am writing to confirm my attendance at tomorrow's meeting. Please let me know if there is anything specific you would like me to prepare or bring to the discussion.\n" +
    '\n' +
    'Thank you, and I look forward to the meeting.\n' +
    '\n' +
    'Best regards,\n' +
    '[Your Name]'
}


Routing between nodes can be done [conditionally](https://langchain-ai.github.io/langgraphjs/concepts/low_level/#conditional-edges) using a simple function.

The return value of this function is used as the name of the node (or list of nodes) to send the state to next.

You can optionally provide a mapping object that maps the output of your router function to the name of the next node.

In [63]:
import { z } from "zod";
import { StateGraph, START, END } from "@langchain/langgraph";


// Define the MessagesState schema
const MessagesState = z.object({
  messages: z.array(z.any()), // Use a more specific type if available
});


// Node: call_llm
const callLlm = async (state) => {
  const output = await modelWithTools.invoke(state.messages);
  return { messages: [output] };
};

// Node: run_tool
const runTool = async (state) => {
  const lastMessage = state.messages[state.messages.length - 1] ;
  const result = [];
  if (lastMessage.tool_calls) {
    for (const toolCall of lastMessage.tool_calls) {
      const observation = await writeEmail.func(toolCall.args);
      result.push({
        type: "tool",
        content: observation,
        tool_call_id: toolCall.id,
      });
    }
  }
  return { messages: result };
};

// Conditional router
const shouldContinue = (state) => {
  const messages = state.messages;
  const lastMessage = messages[messages.length - 1] ;
  if (lastMessage.tool_calls && lastMessage.tool_calls.length > 0) {
    return "run_tool";
  }
  return END;
};

// Build the workflow
const messagesWorkflow = new StateGraph(MessagesState)
  .addNode("call_llm", callLlm)
  .addNode("run_tool", runTool)
  .addEdge(START, "call_llm")
  .addConditionalEdges("call_llm", shouldContinue, { run_tool: "run_tool", [END]: END })
  .addEdge("run_tool", END);

const messagesApp = messagesWorkflow.compile();

In [64]:
// Run the workflow (visualization is not included in TypeScript)
// Example invocation:
const messagesResult = await messagesApp.invoke({
    messages: [
      { type: "human", content: "Draft a response to my boss about tomorrow's meeting" }
    ]
  });
  console.log(messagesResult);

{
  messages: [
    AIMessage {
      "id": "chatcmpl-BWdI3KFRLBe04VNrAZlg3aQbe0bEP",
      "content": "Of course! Could you please provide a bit more detail? For example:\n\n- What is the purpose of the meeting?\n- Do you want to confirm your attendance, ask to reschedule, or provide any specific information?\n- Is there a particular tone you’d like (formal, friendly, brief, etc.)?\n\nLet me know so I can tailor the response for you!",
      "additional_kwargs": {},
      "response_metadata": {
        "tokenUsage": {
          "promptTokens": 61,
          "completionTokens": 77,
          "totalTokens": 138
        },
        "finish_reason": "stop",
        "model_name": "gpt-4.1-2025-04-14",
        "usage": {
          "prompt_tokens": 61,
          "completion_tokens": 77,
          "total_tokens": 138,
          "prompt_tokens_details": {
            "cached_tokens": 0,
            "audio_tokens": 0
          },
          "completion_tokens_details": {
            "reasoning_to

With these low-level components, you can build many different workflows and agents. See the [workflows tutorial](https://langchain-ai.github.io/langgraphjs/tutorials/workflows/) for more examples!

Because agents are such a common pattern, [LangGraph](https://langchain-ai.github.io/langgraphjs/tutorials/workflows/#pre-built) provides a [pre-built agent abstraction](https://langchain-ai.github.io/langgraphjs/agents/overview/).

With LangGraph's [pre-built method](https://langchain-ai.github.io/langgraphjs/how-tos/create-react-agent/), you just pass in the LLM, tools, and prompt.

In [65]:
import { createReactAgent } from "@langchain/langgraph/prebuilt";

// Create a prebuilt React agent
const agent = createReactAgent({
  llm: llm,
  tools: [writeEmail],
  prompt: "Respond to the user's request using the tools provided.",
});

// Run the agent
const agentResult = await agent.invoke({
  messages: [{ type: "human", content: "Draft a response to my boss about tomorrow's meeting" }],
});
console.log(agentResult);

{
  messages: [
    HumanMessage {
      "id": "5cf3d1e5-98f6-4736-9340-2e1edafe6cad",
      "content": "Draft a response to my boss about tomorrow's meeting",
      "additional_kwargs": {},
      "response_metadata": {}
    },
    AIMessage {
      "id": "chatcmpl-BWdI5dHpSyQKh6tejTUexRgmHHRa5",
      "content": "Of course! Could you please provide a bit more detail? For example:\n\n- What is the purpose or topic of the meeting?\n- Do you want to confirm your attendance, ask to reschedule, or address any specific points?\n- Is there a particular tone you’d like (formal, friendly, etc.)?\n\nLet me know so I can tailor the response to your needs!",
      "additional_kwargs": {},
      "response_metadata": {
        "tokenUsage": {
          "promptTokens": 71,
          "completionTokens": 78,
          "totalTokens": 149
        },
        "finish_reason": "stop",
        "model_name": "gpt-4.1-2025-04-14",
        "usage": {
          "prompt_tokens": 71,
          "completion_tokens"

### Persistence

It can be very useful to allow agents to pause and gather human feedback.

LangGraph has a built-in persistence layer, implemented through [checkpointers](https://langchain-ai.github.io/langgraphjs/concepts/persistence/#checkpoints), to enable this.

When you compile a graph with a checkpointer, the checkpointer saves a checkpoint of the graph state at every step.

Checkpoints are saved to a thread, which can be accessed after graph execution.

![checkpointer](img/checkpoints.png)

In [66]:
import { MemorySaver } from "@langchain/langgraph"; 


// Create a React agent with in-memory checkpointing
const agentWithCheckpoint = createReactAgent({
  llm: llm,
  tools: [writeEmail],
  prompt: "Respond to the user's request using the tools provided.",
  checkpointer: new MemorySaver(),
});

// Config for thread id
const config = { configurable: { thread_id: "1" } };

// Run the agent with checkpointing
const checkpointResult = await agentWithCheckpoint.invoke(
  {
    messages: [{ type: "human", content: "What are some good practices for writing emails?" }],
  },
  config
);
console.log(checkpointResult);

{
  messages: [
    HumanMessage {
      "id": "75a463e5-eaf1-43b4-8c9f-2c739bc76c41",
      "content": "What are some good practices for writing emails?",
      "additional_kwargs": {},
      "response_metadata": {}
    },
    AIMessage {
      "id": "chatcmpl-BWdI7Sv1ENX9IC8ZKkqaWexB1zPZO",
      "content": "Here are some good practices for writing effective emails:\n\n1. **Use a Clear Subject Line:** Make your subject concise and informative so the recipient knows what the email is about.\n\n2. **Greet Appropriately:** Start with a polite greeting, such as “Hello [Name],” or “Dear [Name],” depending on the formality.\n\n3. **Be Concise and to the Point:** State your purpose early and keep your message brief. Avoid unnecessary details.\n\n4. **Use Proper Grammar and Spelling:** Proofread your email to ensure it is free of errors and maintains a professional tone.\n\n5. **Structure Your Email:** Use short paragraphs, bullet points, or numbered lists to make your message easy to read.\

In [67]:
// Get the latest state snapshot for the thread
const state = await agentWithCheckpoint.getState(config);
for (const message of state.values.messages) {
  // You can define a prettyPrint function or just log the message
  console.log(message);
}

HumanMessage {
  "id": "75a463e5-eaf1-43b4-8c9f-2c739bc76c41",
  "content": "What are some good practices for writing emails?",
  "additional_kwargs": {},
  "response_metadata": {}
}
AIMessage {
  "id": "chatcmpl-BWdI7Sv1ENX9IC8ZKkqaWexB1zPZO",
  "content": "Here are some good practices for writing effective emails:\n\n1. **Use a Clear Subject Line:** Make your subject concise and informative so the recipient knows what the email is about.\n\n2. **Greet Appropriately:** Start with a polite greeting, such as “Hello [Name],” or “Dear [Name],” depending on the formality.\n\n3. **Be Concise and to the Point:** State your purpose early and keep your message brief. Avoid unnecessary details.\n\n4. **Use Proper Grammar and Spelling:** Proofread your email to ensure it is free of errors and maintains a professional tone.\n\n5. **Structure Your Email:** Use short paragraphs, bullet points, or numbered lists to make your message easy to read.\n\n6. **Be Polite and Courteous:** Use polite languag

In [ ]:
// Get the latest state snapshot for the thread
const state = await agentWithCheckpoint.getState(config);
for (const message of state.values.messages) {
  console.log(message);
}

HumanMessage {
  "id": "75a463e5-eaf1-43b4-8c9f-2c739bc76c41",
  "content": "What are some good practices for writing emails?",
  "additional_kwargs": {},
  "response_metadata": {}
}
AIMessage {
  "id": "chatcmpl-BWdI7Sv1ENX9IC8ZKkqaWexB1zPZO",
  "content": "Here are some good practices for writing effective emails:\n\n1. **Use a Clear Subject Line:** Make your subject concise and informative so the recipient knows what the email is about.\n\n2. **Greet Appropriately:** Start with a polite greeting, such as “Hello [Name],” or “Dear [Name],” depending on the formality.\n\n3. **Be Concise and to the Point:** State your purpose early and keep your message brief. Avoid unnecessary details.\n\n4. **Use Proper Grammar and Spelling:** Proofread your email to ensure it is free of errors and maintains a professional tone.\n\n5. **Structure Your Email:** Use short paragraphs, bullet points, or numbered lists to make your message easy to read.\n\n6. **Be Polite and Courteous:** Use polite languag

In [69]:
// Continue the conversation again
const finalResult = await agentWithCheckpoint.invoke(
    {
      messages: [
        { type: "human", content: "I like this, let's write the email" },
      ],
    },
    config
  );
  for (const m of finalResult.messages) {
    console.log(m);
  }

HumanMessage {
  "id": "75a463e5-eaf1-43b4-8c9f-2c739bc76c41",
  "content": "What are some good practices for writing emails?",
  "additional_kwargs": {},
  "response_metadata": {}
}
AIMessage {
  "id": "chatcmpl-BWdI7Sv1ENX9IC8ZKkqaWexB1zPZO",
  "content": "Here are some good practices for writing effective emails:\n\n1. **Use a Clear Subject Line:** Make your subject concise and informative so the recipient knows what the email is about.\n\n2. **Greet Appropriately:** Start with a polite greeting, such as “Hello [Name],” or “Dear [Name],” depending on the formality.\n\n3. **Be Concise and to the Point:** State your purpose early and keep your message brief. Avoid unnecessary details.\n\n4. **Use Proper Grammar and Spelling:** Proofread your email to ensure it is free of errors and maintains a professional tone.\n\n5. **Structure Your Email:** Use short paragraphs, bullet points, or numbered lists to make your message easy to read.\n\n6. **Be Polite and Courteous:** Use polite languag

### Testing, Debugging, and Deployment

When using LangChain or LangGraph JS, [LangSmith logging works out of the box](https://docs.smith.langchain.com/observability/how_to_guides/trace_with_langgraph) by setting the following environment variables:


In [70]:
// Set LangSmith tracing environment variables in your .env or process environment
// Example (do this in your shell or .env file, not in code):
// process.env.LANGSMITH_TRACING = "true";
// process.env.LANGSMITH_API_KEY = "<your-langsmith-api-key>";

Here is the LangSmith trace from above graph execution:

https://smith.langchain.com/public/6f77014f-d054-44ed-aa2c-8b06ceab689f/r

We can see that the agent is able to continue the conversation from the previous state because we used a checkpointer.

It's also easy to deploy to deploy our graph using [LangGraph Platform](https://langchain-ai.github.io/langgraphjs/concepts/langgraph_platform/). 

We simply need to ensure our project has [a structure](https://langchain-ai.github.io/langgraphjs/cloud/deployment/setup_javascript/) like this:

```
my-app/
├── my_agent # all project code lies within here
│   └── agent.ts # code for constructing your graph
├── .env # environment variables
├── langgraph.json  # configuration file for LangGraph
└── pyproject.toml # dependencies for your project
```

The `langgraph.json` file specifies the dependencies, graphs, environment variables, and other settings required to deploy a LangGraph application.

Note: Langgraph 101 script for deployment is not available in typescript, however you can deploy `email_assistant.ts` `email_assistant_hitl.ts` and `email_assistant_hitl_memory.ts`

There are a range of [deployment options](https://langchain-ai.github.io/langgraphjs/tutorials/deployment/). 

* All create an API [server](https://langchain-ai.github.io/langgraphjs/concepts/langgraph_server/) for our graph
* All include an interactive IDE (LangGraph [Studio](https://langchain-ai.github.io/langgraphjs)).
 
We can start a deployment locally using `langgraph dev`:

Here we can see a visualization of the graph as well as the graph state in Studio.

![langgraph_studio](img/langgraph_studio.png)